# SQL con clase

*The single biggest challenge to learning SQL programming is unlearning procedureal programming.*
-- Joe Celko

En este notebook, exploraremos diversos estilos y prácticas para escribir consultas SQL enfocadas en el análisis de datos. Las consultas analíticas se distinguen de las consultas convencionales en que están diseñadas específicamente para trabajar sobre grandes volúmenes de datos y tienen a ser más complejas.

## Setup

Primero vamos a configurar la notebook y definir las variables principales.

Instalamos los paquetes necesarios. Este paso se puede omitir si ya lo realizó en su entorno, sin embargo si usa **Colab** va a ser necesario ejecutarlo.

In [1]:
%%bash
pip install duckdb
pip install jupysql
pip install duckdb-engine

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


Configuracion la extensión de %sql para el notebook, lo que nos va a permitir usar lenguaje SQL directamente sobre una celda

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

ModuleNotFoundError: No module named 'pandas'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = '/content/drive/MyDrive/Data Mining/'
dataset_file = 'competencia_01_crudo.csv'

Y finalmente cargamos el dataset en la base de datos

In [ ]:
%%sql
create or replace table competencia_01_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


## WITH



Un escenario común es necesitar realizar una consulta a no a una tabla, sino a otra consulta. Una forma tradicional de hacerlo es usando es:
* Creando una tabla temporal con esa consulta
* Usando `selects from select`.

Esta necesidad es muy común, pero como todo en la vida hay peros.
- Crear una tabla en una base de datos que va a ser usada UNA SOLA VEZ ensucia los esquemas con un montón de tablas que no son útiles y además del desorden ocupan espacio innecesario
- El `select from select`, debe ser de las cosas peor vistas por los administradores de BBDD. Tienen sus razones técnicas, en general la recomendación es no usarlo a menos que uno conozca bien como funciona la base de datos por dentro, ya que hay casos donde terminan ocasionando muchos problemas

Y que hacer? bueno hay una clausula en **SQL** llamada `WITH`, que nos permite crear esas (plural) tablas para ser usadas en la **query** principal. Ejemplo si queremos comparar el valor de una variable con su promedio a través de todos los meses.


In [ ]:
%%sql
with promedios as (
    select
        numero_de_cliente
        , avg(mrentabilidad) as avg_mrentabilidad
    from competencia_01_crudo
    group by numero_de_cliente
) select
    cp.numero_de_cliente
    , cp.foto_mes
    , if(cp.mrentabilidad > p.avg_mrentabilidad, 'mayor', 'menor') as comparacion
from competencia_01_crudo cp
join promedios p using (numero_de_cliente)


,numero_de_cliente,foto_mes,comparacion
0,918105588,202101,mayor
1,918119003,202101,menor
2,918123743,202101,menor
3,918123965,202101,mayor
4,918147740,202101,mayor
...,...,...,...
978434,1598368433,202106,menor
978435,1598419415,202106,menor
978436,1598425905,202106,menor
978437,1598470389,202106,menor


- Usar esa clausula tiene muchos beneficios, entre ellos sumar legibilidad de código y le permite al motor de **SQL** optimizar mejor, ya que entiende todo lo que se buscar hacer y planifica como resolverlo en el menor tiempo posible.

**NOTA**: Hay un tipo de tabla que se llaman temporales, que se borran solas una vez que uno se desconecta de la base de datos. Es una mejor alternativa frente al `CREATE`, sin embargo si la tabla que se esta creando se usa una sola vez es conveniente usar el `WITH` porque permite optimizar la ejecución. Si esa tabla va a ser usada mas de una vez dentro del proceso, las tablas temporales son la mejor opción.

# Funciones Analíticas

Las funciones analíticas en SQL son un conjunto de funciones que te permiten realizar cálculos avanzados sobre un conjunto de filas relacionadas dentro de una consulta conservando las filas individuales y calculando sobre ventanas de datos relacionadas.

Veamos un ejemplo para que quede más claro:

In [ ]:
%%sql
select
    numero_de_cliente
    , foto_mes
    , mrentabilidad
from competencia_01_crudo
where numero_de_cliente = 918147740
order by foto_mes

,numero_de_cliente,foto_mes,mrentabilidad
0,918147740,202101,276.74
1,918147740,202102,-735.18
2,918147740,202103,-417.20
3,918147740,202104,-694.08
4,918147740,202105,-1692.76
5,918147740,202106,-897.20


In [ ]:
%%sql
select
    numero_de_cliente
    , foto_mes
    , mrentabilidad
    , lead(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_mas_1
    , lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_menos_1
    , lead(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_mas_2
    , lag(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as mrentabilidad_menos_2
from competencia_01_crudo
where numero_de_cliente = 918147740
order by foto_mes



,numero_de_cliente,foto_mes,mrentabilidad,mrentabilidad_mas_1,mrentabilidad_menos_1,mrentabilidad_mas_2,mrentabilidad_menos_2
0,918147740,202101,276.74,-735.18,NaN,-417.20,NaN
1,918147740,202102,-735.18,-417.20,276.74,-694.08,NaN
2,918147740,202103,-417.20,-694.08,-735.18,-1692.76,276.74
3,918147740,202104,-694.08,-1692.76,-417.20,-897.20,-735.18
4,918147740,202105,-1692.76,-897.20,-694.08,NaN,-417.20
5,918147740,202106,-897.20,NaN,-1692.76,NaN,-694.08


In [ ]:
%%sql
SELECT row_number() OVER (order by foto_mes)
FROM competencia_01_crudo;

,row_number() OVER (ORDER BY foto_mes)
0,1
1,2
2,3
3,4
4,5
...,...
978434,978435
978435,978436
978436,978437
978437,978438


In [ ]:
%%sql
SELECT numero_de_cliente,foto_mes, row_number() OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as rn
FROM competencia_01_crudo;

,numero_de_cliente,foto_mes,rn
0,249320580,202101,1
1,249320580,202102,2
2,249320580,202103,3
3,249320580,202104,4
4,249320580,202105,5
...,...,...,...
978434,1596008221,202106,1
978435,1596632988,202106,1
978436,1596681776,202106,1
978437,1596941179,202106,1


In [ ]:
%%sql
SELECT numero_de_cliente,
foto_mes,
mrentabilidad,
avg(mrentabilidad) OVER (PARTITION BY numero_de_cliente) as prm, --con el order by es promedio acumulado
row_number() OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as rn,
FROM competencia_01_crudo;

,numero_de_cliente,foto_mes,mrentabilidad,prm,rn
0,249320580,202101,508.80,2712.216667,1
1,249320580,202102,2494.79,2712.216667,2
2,249320580,202103,3207.66,2712.216667,3
3,249320580,202104,3122.15,2712.216667,4
4,249320580,202105,3455.86,2712.216667,5
...,...,...,...,...,...
978434,1596008221,202106,111.32,111.320000,1
978435,1596632988,202106,114.68,114.680000,1
978436,1596681776,202106,99.47,99.470000,1
978437,1596941179,202106,6762.13,6762.130000,1


**sin** **order** **by**

"Para cada fila, calculá el promedio de mrentabilidad considerando todas las filas de la partición (todas las del mismo numero_de_cliente)."

* El resultado es constante dentro de cada partición, porque todas las filas miran el mismo conjunto completo.

**con order by**
le estás diciendo al motor:

"Para cada fila, calculá el promedio de mrentabilidad considerando las filas de la partición desde el inicio hasta la fila actual, siguiendo el orden de foto_mes."

* Es decir, ya no toma todas las filas de la partición, sino un subconjunto que depende de la fila actual → eso se llama ventana creciente o ventana acumulada (running window).

* Qué paso?
* ¿Cómo esta construyendo las nuevas variables?
* ¿ Nos puede ayudar a construir targets ?

¿Dondé puedo leer un poco más de esta magia negra? https://duckdb.org/docs/sql/window_functions.html

## Los casos raros

Que pasa con el cliente **930544172** y la foto **202103**


In [ ]:
%%sql
select
    numero_de_cliente
    , foto_mes
from competencia_01_crudo
where numero_de_cliente = 930544172
order by foto_mes

,numero_de_cliente,foto_mes
0,930544172,202101
1,930544172,202102


* ¿Cuál debería ser la clase para cada periodo?

### Una pista

Podemos generar todas las posibles combinaciones de clientes y periodos de manera muy simple

In [ ]:
%%sql
with periodos as (
  select distinct foto_mes from competencia_01_crudo
), clientes as (
  select distinct numero_de_cliente from competencia_01_crudo
)
select numero_de_cliente, foto_mes from clientes cross join periodos
where numero_de_cliente = 930544172
order by foto_mes

,numero_de_cliente,foto_mes
0,930544172,202101
1,930544172,202102
2,930544172,202103
3,930544172,202104
4,930544172,202105
5,930544172,202106


Solo nos queda saber si estuvo o no el banco el cliente en ese periodo y armar el

In [ ]:
%%sql
with periodos as (
    select distinct foto_mes from competencia_01_crudo -- Esto también se puede hacer con secuencias
), clientes as (
    select distinct numero_de_cliente from competencia_01_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
)
select
    t.numero_de_cliente
    , t.foto_mes
    , if(c.numero_de_cliente is null, 0, 1) as mes_0
from todo t
left join competencia_01_crudo c using (numero_de_cliente, foto_mes)
where t.numero_de_cliente = 930544172
order by foto_mes

,numero_de_cliente,foto_mes,mes_0
0,930544172,202101,1
1,930544172,202102,1
2,930544172,202103,0
3,930544172,202104,0
4,930544172,202105,0
5,930544172,202106,0


Antes de continuar, tomémonos un momento para reflexionar. Con todas estas piezas, ¿cómo podemos ensamblar el target?

Una vez que lo tengamos claro sobre el papel, proceda a completar el código que sigue.

In [ ]:
%%sql
create or replace table competencia_01 as
with periodos as (
    select distinct foto_mes from competencia_01_crudo
), clientes as (
    select distinct numero_de_cliente from competencia_01_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , null AS clase_ternaria-- Replazar null por la lógica que genera el target
    from todo t
    left join competencia_01_crudo c using (numero_de_cliente, foto_mes)
) select
  * EXCLUDE (mes_0, mes_1, mes_2) --excluye meses
from clase_ternaria
where mes_0 = 1

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Revisamos que todo salga como esperamos

In [ ]:
%sql select count(*) from competencia_01

,count_star()
0,978439


In [ ]:
%%sql
select * from competencia_01

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249378334,202101,1,0,0,49,270,9725.71,61292.37,5316.83,...,4,0.0,-150900.35,0.0,7957,61083.61,18,0,4914.87,<NA>
1,249378334,202102,1,0,0,49,271,3805.08,61991.99,2396.27,...,-3,0.0,-86611.91,0.0,7985,35054.88,12,0,3167.10,<NA>
2,249378334,202103,1,0,0,49,272,4287.70,62511.34,2967.34,...,1,0.0,-56070.67,0.0,8016,79505.02,23,0,5149.47,<NA>
3,249378334,202104,1,0,0,50,273,3143.45,59556.76,3070.33,...,2,0.0,-103864.39,0.0,8046,83225.00,17,0,5161.20,<NA>
4,249378334,202105,1,0,0,50,274,2889.40,56888.45,2767.35,...,5,0.0,-106747.30,0.0,8077,89641.29,17,0,7084.92,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,1583149218,202102,0,0,0,33,1,0.00,0.00,0.00,...,-3,0.0,NaN,NaN,3,NaN,<NA>,<NA>,0.00,<NA>
978435,1583149218,202103,1,0,0,33,2,353.06,353.06,11.35,...,1,0.0,0.00,0.0,34,0.00,0,0,0.00,<NA>
978436,1583149218,202104,1,0,0,33,3,764.88,1117.95,30.88,...,2,0.0,0.00,0.0,64,0.00,0,0,0.00,<NA>
978437,1583149218,202105,1,0,0,33,4,-5413.86,-4295.91,8.01,...,5,0.0,0.00,0.0,95,0.00,0,0,0.00,<NA>


Y vemos la cardinalidad de las clases por periodo

In [ ]:
%%sql
PIVOT competencia_01
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes
0,202101
1,202102
2,202103
3,202104
4,202105
5,202106


Y finalmente almacenamos el nuevo dataset para ser usado a continuación

In [ ]:
%%sql COPY competencia_01 TO '{{dataset_path}}competencia_01.csv' (FORMAT CSV, HEADER)


## El pato del amor

DuckDB no es una base de datos SQL estándar; su sintaxis moderna facilita significativamente la escritura de consultas. A continuación, te menciono algunas características destacadas:

+ Uso de variables: En DuckDB, no es necesario reescribir las variables; puedes utilizar tus variables una vez definidas, lo que simplifica el código.

+ Función IF: DuckDB soporta la función IF(condición, valor_si_verdadero, valor_si_falso), lo que hace que tu código sea más claro y legible en comparación con el uso de CASE WHEN.

+ Exclusión de campos: Puedes utilizar SELECT * EXCLUDE(field1, ...) para excluir campos específicos de la lista de selección, lo que ahorra tiempo y esfuerzo.

+ Alias en cláusulas: Es posible utilizar alias en las cláusulas WHERE, GROUP BY y HAVING, lo que añade flexibilidad a la escritura de consultas.

+ Uso de alias en cálculos: DuckDB permite el uso de alias dentro del cálculo de otros campos, facilitando la manipulación y transformación de datos en una consulta.

Estas son solo algunas de las capacidades que hacen de DuckDB una herramienta poderosa. Te recomiendo leer la documentación y el blog oficial para explorar más a fondo su potencial.